In [ ]:
%matplotlib inline
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from preparation.data import Data
import h5py
import os
import shutil

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
def leakyReLU(x, alpha):
    return tf.maximum(alpha*x, x)

In [ ]:
# Simple function to plot number images.
def plot_images(plt_num, images, dim):
    # Standard parameters for the plot.
    
    mpl.rcParams["figure.figsize"] = dim, dim
    fig = plt.figure()
    for i in range(0, plt_num):
        fig.add_subplot(1, 10, i+1)
        img = images[i, :, :, :]
        plt.imshow(img)
    plt.show()

In [ ]:
data = Data(dataset='nki', marker='he', patch_h=112, patch_w=112, n_channels=3, batch_size=10)
images, label = data.training.next_batch(10)

plot_images(plt_num=10, images=images, dim=20)

In [ ]:
def model_inputs(image_width, image_height, image_channels, z_dim):
    real_images_56 = tf.placeholder(dtype=tf.float32, shape=(None, image_width, image_height, image_channels), name='real_images_56')
    real_images_112 = tf.placeholder(dtype=tf.float32, shape=(None, 2*image_width, 2*image_height, image_channels), name='real_images_112')
    z_input_56 = tf.placeholder(dtype=tf.float32, shape=(None, z_dim), name='z_input_56')
    z_input_112 = tf.placeholder(dtype=tf.float32, shape=(None, z_dim), name='z_input_112')
    learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
    return real_images_56, real_images_112, z_input_56, z_input_112, learning_rate

In [ ]:
def feature_normalization(inputs, normalize):
    epsilon = 1e-8
    normalization = 1.0
    if normalize:
        # normalization done accross channels
        pixels_mean = tf.reduce_mean(tf.square(inputs), axis=-1, keep_dims=true)
        normalization = 1.0/tf.sqrt(pixels_mean + epsilon)
    return inputs*normalization


# In here we combine Nearest neighbor interpolation and conv, it is faster than upscaling by nearest neirbor 
# and then conv.
def convolutional(inputs, filter_size, output_channels, stride, padding, conv_type, output_shape):
    
    current_shape = inputs.get_shape()
    input_channels = current_shape[3]
    
    weight_init = tf.contrib.layers.xavier_initializer()
    filter_shape = (filter_size, filter_size, input_channels, output_channels)    
    filter = tf.get_variable('filter', filter_shape, initializer=weight_init)    
    b = tf.get_variable('bias', [1, 1, 1, out_channels], initializer=tf.constant_initializer(0))
    
    if conv_type == 'upscale':
        output_shape = (current_shape[0], current_shape[1]*2, current_shape[2]*2, current_shape[3])
        # Weight filter initializer.
        filter = tf.pad(filter, ([1,1], [1,1], [0,0], [0,0]), mode='CONSTANT')
        filter = tf.add_n([filter[1:,1:], filter[:-1,1:], filter[1:,:-1], filter[:-1,:-1]])
        strides = (1, 1, 2, 2)
        output = tf.nn.conv2d_transpose(inputs, filter, output_shape, strides, padding)
        
    elif conv_type == 'downscale':
        # Weight filter initializer.
        filter = tf.pad(filter, ([1,1], [1,1], [0,0], [0,0]), mode='CONSTANT')
        filter = tf.add_n([filter[1:,1:], filter[:-1,1:], filter[1:,:-1], filter[:-1,:-1]])
        strides = (1, 1, 2, 2)
        output = tf.nn.conv2d(inputs, filter, strides, padding)
        
    elif conv_type == 'transpose':
        strides = (1, 1, stride, stride)
        output = tf.nn.conv2d_transpose(inputs, filter, output_shape, strides, padding)
    
    elif conv_type == 'convolutional':
        strides = (1, 1, stride, stride)
        output = tf.nn.conv2d(inputs, filter, strides, padding)
    
    output += b
    
    return output
    

def generator_layer(inputs, filter_size, output_channels, stride, padding, alpha, normalize, conv_type, output_shape=None):
    # Conv.
    net = convolutional_t(inputs, filter_size, output_channels, stride, padding, conv_type, output_shape)

    # LReLU or Linear.
    if alpha not is None:
        net = leakyReLU(net, alpha)

    # Pixelwise_norm.
    net = feature_normalization(net, normalize)
    return net

def discriminator_layer(inputs, filter_size, output_channels, stride, padding, alpha, conv_type):
    # Conv.
    net = convolutional(inputs, filter_size, output_channels, stride, padding, conv_type)

    # LReLU or Linear.
    if alpha not is None:
        net = leakyReLU(net, alpha)

    return net

In [ ]:
def generator(z_input, num_layers, channels):
    with tf.variable_scope('generator'):
        with tf.variable_scope('latent'):
            net = tf.expand_dims(z_input, 2)
            net = tf.expand_dims(z_input, 3)

        for i in range(num_layers):
            with tf.variable_scope('layer_%s' % i):
                # First block layer
                if i == num_layers-1:
                    net_prev_block = net
                elif i == 0:
                    # First Conv4 LReLU
                    with tf.variable_scope('conv4_1'):
                        output_shape = (net.get_shape[0], 4, 4, channels[i])
                        net = generator_layer(net, filter_size=4, output_channels=channels[i], stride=1, padding='VALID', 
                                              alpha=0.2, normalize=True, conv_type='transpose')
                    # Second Conv3 LReLU
                    with tf.variable_scope('conv3_2'):
                        net = generator_layer(net, filter_size=3, output_channels=channels[i], stride=1, padding='SAME', 
                                              alpha=0.2, normalize=True, conv_type='convolutional')
                else:
                    # Upsample x2
                    with tf.variable_scope('upsample_1'):
                        net = generator_layer(net, filter_size=3, output_channels=channels[i], stride=2, padding='SAME', 
                                              alpha=None, normalize=True, conv_type='upscale')
                    # First Conv3 LReLU
                    with tf.variable_scope('conv3_2'):
                        net = generator_layer(net, filter_size=3, output_channels=channels[i], stride=1, padding='SAME', 
                                              alpha=0.2, normalize=True, conv_type='convolutional')
                    # Second Conv3 LReLU
                    with tf.variable_scope('conv3_3'):
                        net = generator_layer(net, filter_size=3, output_channels=channels[i], stride=1, padding='SAME', 
                                              alpha=0.2, normalize=True, conv_type='convolutional')
        # Third Conv1 linear.           
        with tf.variable_scope('RGB_layer_%s' % i):
            net = generator_layer(net, filter_size=1, output_channels=3, stride=1, padding='SAME', 
                                  alpha=None, normalize=True, conv_type='convolutional')

In [ ]:
def discriminator(z_input, num_layers, channels):
    with tf.variable_scope('discriminator'):
        
        # RGB Transition.
        
        for i in range(num_layers):
            with tf.variable_scope('layer_%s' % i):
                if i==0:
                    # Minibatch.
                else:
                    # First Conv3 LReLU
                    with tf.variable_scope('conv3_1'):
                        net = discriminator_layer(net, filter_size=3, output_channels=channels[i], stride=1, padding='SAME', 
                                              alpha=0.2, normalize=True, conv_type='convolutional')
                    # Second Conv3 LReLU
                    with tf.variable_scope('conv3_2'):
                        net = discriminator_layer(net, filter_size=3, output_channels=channels[i], stride=1, padding='SAME', 
                                              alpha=0.2, normalize=True, conv_type='convolutional')
                    # Downsample /2
                    with tf.variable_scope('downsample_3'):
                        net = discriminator_layer(net, filter_size=3, output_channels=channels[i], stride=2, padding='SAME', 
                                              alpha=None, conv_type='downscale')
                    
            
    

In [ ]:
def show_generated(session, output_fake, n_images, z_input_56, z_input_112):
    z_dim = z_input_56.get_shape()[-1]
    sample_z_56 = np.random.uniform(low=-1., high=1., size=(n_images, z_dim))
    sample_z_112 = np.random.uniform(low=-1., high=1., size=(n_images, z_dim))
    feed_dict = {z_input_56:sample_z_56, z_input_112:sample_z_112}
    gen_samples = session.run(output_fake, feed_dict=feed_dict)
    plot_images(plt_num=n_images, images=gen_samples, dim=20)    
    return gen_samples, sample_z_56

In [ ]:
def save_loss(losses, data_out_path, dim):    
    mpl.rcParams["figure.figsize"] = dim, dim
    plt.rcParams.update({'font.size': 22})
    losses = np.array(losses)
    fig, ax = plt.subplots()
    plt.plot(losses[:, 0], label='Discriminator', alpha=0.5)
    plt.plot(losses[:, 1], label='Generator', alpha=0.5)
    plt.title("Training Losses")
    plt.legend()
    plt.savefig('%s/training_loss.png' % data_out_path)

In [ ]:
def setup_output(show_epochs, epochs, data, n_images, image_height, image_width, image_channels, z_dim, data_out_path):
    checkpoints_path = os.path.join(data_out_path, 'checkpoints')
    checkpoints = os.path.join(checkpoints_path, 'DCGAN.ckt')
    gen_images_path = os.path.join(data_out_path, 'images')
    gen_images = os.path.join(gen_images_path, 'gen_images.h5')
    latent_images = os.path.join(gen_images_path, 'latent_images.h5')
    if os.path.isdir(checkpoints_path):
         shutil.rmtree(checkpoints_path)
    if os.path.isdir(gen_images_path):
         shutil.rmtree(gen_images_path)
    os.makedirs(checkpoints_path)
    os.makedirs(gen_images_path)

    #TODO
    size_img = (epochs*data[0].training.iterations)//show_epochs
    img_db_shape = (size_img, n_images, image_height, image_width, image_channels)
    latent_db_shape = (size_img, n_images, z_dim)
    hdf5_gen = h5py.File(gen_images, mode='w')
    hdf5_latent = h5py.File(latent_images, mode='w')
    img_storage = hdf5_gen.create_dataset(name='generated_img', shape=img_db_shape, dtype=np.float32)
    latent_storage = hdf5_latent.create_dataset(name='generated_img', shape=latent_db_shape, dtype=np.float32)
    return img_storage, latent_storage, checkpoints

In [ ]:
def loss(real_images_56, real_images_112, z_input_56, z_input_112, out_channel_dim, alpha):
    
    # Generator.
    fake_images_56, fake_images_112 = generator(z_input_56=z_input_56, z_input_112=z_input_112, out_channel_dim=out_channel_dim, 
                                                reuse=False, is_train=True, alpha=alpha)
    
    # Discriminator.
    output_fake_56, logits_fake_56, output_fake_112, logits_fake_112 = discriminator(images_56=fake_images_56, images_112=fake_images_112, reuse=False, alpha=alpha) 
    output_real_56, logits_real_56, output_real_112, logits_real_112 = discriminator(images_56=real_images_56, images_112=real_images_112, reuse=True, alpha=alpha)
    
    '''
    Discriminator Losses 56, 112
    '''
    # Discriminator loss 56.
    loss_dis_fake_56 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake_56, labels=tf.zeros_like(output_fake_56)))
    loss_dis_real_56 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_real_56, labels=tf.ones_like(output_fake_56)*0.9))
    loss_dis_56 = loss_dis_fake_56 + loss_dis_real_56
    
    # Discriminator loss 112.
    loss_dis_fake_112 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake_112, labels=tf.zeros_like(output_fake_112)))
    loss_dis_real_112 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_real_112, labels=tf.ones_like(output_fake_112)*0.9))
    loss_dis_112 = loss_dis_fake_112 + loss_dis_real_112

    '''
    Generator Losses 56, 112
    This is where we implement -log[D(G(z))] instead log[1-D(G(z))].
    Recall the implementation of cross-entropy, sign already in. 
    '''
    # Generator loss 56.
    loss_gen_56 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake_56, labels=tf.ones_like(output_fake_56)))

    # Generator loss 112.
    loss_gen_112 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_fake_112, labels=tf.ones_like(output_fake_112)))
    
    losses_56 = [loss_dis_56, loss_gen_56, loss_dis_fake_56, loss_dis_real_56]
    losses_112 = [loss_dis_112, loss_gen_112, loss_dis_fake_112, loss_dis_real_112]

    return losses_56, losses_112

In [ ]:
def optimization(loss_dis, loss_gen, learning_rate, beta1, level):
    trainable_variables = tf.trainable_variables()
    
    generator_variables_def = [variable for variable in trainable_variables if variable.name.startswith('generator/generator_base')]
    generator_variables_res = [variable for variable in trainable_variables if variable.name.startswith('generator/generator_%s'%level)]
    generator_variables = generator_variables_def + generator_variables_res
    
    discriminator_variables_def = [variable for variable in trainable_variables if variable.name.startswith('discriminator/discriminator_base')]
    discriminator_variables_res = [variable for variable in trainable_variables if variable.name.startswith('discriminator/discriminator_%s'%level)]
    discriminator_variables = discriminator_variables_def + discriminator_variables_res
    
#     print('Generator_base: ', generator_variables_def)
#     print('Generator_res%s: ' % level, generator_variables_res)
    
#     print('Discriminator_base: ', discriminator_variables_def)
#     print('Discriminator_res%s: ' % level, discriminator_variables_res)
    
    # Handling Batch Normalization.
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
        train_generator = tf.train.AdamOptimizer(learning_rate=learning_rate, 
                                                 beta1=beta1).minimize(loss_gen, var_list=generator_variables)
        train_discriminator = tf.train.AdamOptimizer(learning_rate=learning_rate, 
                                                     beta1=beta1).minimize(loss_dis, var_list=discriminator_variables)    
    return train_generator, train_discriminator

In [ ]:
def train(epochs, batch_size, z_dim, learning_rate, beta1, data, alpha, image_width, image_height, image_channels, data_out_path):
    # Placeholder for inputs
    real_images_56, real_images_112, z_input_56, z_input_112, learning_rate_input = model_inputs(image_width=image_width, image_height=image_height, 
                                                                           image_channels=image_channels, z_dim=z_dim)
    
    # Define losses and model. 
    losses_56, losses_112 = loss(real_images_56=real_images_56, real_images_112=real_images_112, z_input_56=z_input_56, z_input_112=z_input_112, 
                                 out_channel_dim=image_channels, alpha=alpha)
    loss_dis_56, loss_gen_56, loss_dis_fake_56, loss_dis_real_56 = losses_56
    loss_dis_112, loss_gen_112, loss_dis_fake_112, loss_dis_real_112 = losses_112
    
    # Optimizations for 56 and 112 resolutions.
    train_generator_56, train_discriminator_56 = optimization(loss_dis=loss_dis_56, loss_gen=loss_gen_56, learning_rate=learning_rate_input, beta1=beta1,
                                                             level='56')
    train_generator_112, train_discriminator_112 = optimization(loss_dis=loss_dis_112, loss_gen=loss_gen_112, learning_rate=learning_rate_input, beta1=beta1,
                                                               level='112')
    
    # Generators to produce images.
    output_gen_56, output_gen_112 = generator(z_input_56=z_input_56, z_input_112=z_input_112, out_channel_dim=image_channels, reuse=True, is_train=False, alpha=alpha)

    global losses_track_56
    global losses_track_112
        
    show_epochs = 100
    print_epochs = 10
    n_images = 10
        
    losses_track_56 = list()
    losses_track_112 = list()
    
    data_56, data_112 = data

    #Saver for model weights/bias, and generated images.
    saver = tf.train.Saver()
    
    img_storage, latent_storage, checkpoints = setup_output(show_epochs, epochs, data, n_images, image_height, image_width, image_channels, z_dim, data_out_path)
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        
        '''
        First level training for 56x56x3.
        '''
        run_epochs = 0
        for epoch in range(1, epochs+1):
            for batch_images, batch_labels in data_56.training:
                z_batch = np.random.uniform(low=-1., high=1., size=(batch_size, z_dim))
                real_112 = np.zeros((batch_size, 112, 112, 3))
                z_ba_112 = np.zeros((batch_size, z_dim))
                feed_dict = {z_input_56:z_batch, z_input_112:z_ba_112, real_images_56:batch_images, real_images_112:real_112, learning_rate_input: learning_rate}
                session.run(train_discriminator_56, feed_dict=feed_dict)
                session.run(train_generator_56, feed_dict=feed_dict)
               
                if run_epochs%print_epochs==0:
                    feed_dict = {z_input_56:z_batch, z_input_112:z_ba_112, real_images_56:batch_images, real_images_112:real_112}
                    epoch_loss_dis = session.run(loss_dis_56, feed_dict=feed_dict)
                    epoch_loss_gen = session.run(loss_gen_56, feed_dict=feed_dict)
                    losses_track_56.append((epoch_loss_dis, epoch_loss_gen))
                    print('Epochs %s/%s: Generator Loss: %s. Discriminator Loss: %s' % (epoch, epochs, np.round(epoch_loss_gen, 3), np.round(epoch_loss_dis, 3)))
                if run_epochs%show_epochs == 0:
                    gen_samples, sample_z = show_generated(session=session, output_fake=output_gen_56, n_images=n_images, z_input_56=z_input_56, z_input_112=z_input_112)
#                     img_storage[run_epochs//show_epochs] = gen_samples
#                     latent_storage[run_epochs//show_epochs] = sample_z
                    saver.save(sess = session, save_path=checkpoints, global_step=run_epochs)
                
                run_epochs+=1
        '''
        Second level training for 112x112x3.
        '''
        for epoch in range(1, epochs+1):
            for batch_images, batch_labels in data_112.training:
                z_batch = np.random.uniform(low=-1., high=1., size=(batch_size, z_dim))
                real_56 = np.zeros((batch_size, 112, 112, 3))
                z_ba_56 = np.zeros((batch_size, z_dim))
                feed_dict = {z_input_56:z_ba_56, z_input_112:z_batch, real_images_56:real_56, real_images_112:batch_images, learning_rate_input: learning_rate}
                session.run(train_discriminator_112, feed_dict=feed_dict)
                session.run(train_generator_112, feed_dict=feed_dict)
               
                if run_epochs%print_epochs==0:
                    feed_dict = {z_input_56:z_ba_56, z_input_112:z_batch, real_images_56:real_56, real_images_112:batch_images}
                    epoch_loss_dis = session.run(loss_dis_112, feed_dict=feed_dict)
                    epoch_loss_gen = session.run(loss_gen_112, feed_dict=feed_dict)
                    losses_track_112.append((epoch_loss_dis, epoch_loss_gen))
                    print('Epochs %s/%s: Generator Loss: %s. Discriminator Loss: %s' % (epoch, epochs, np.round(epoch_loss_gen, 3), np.round(epoch_loss_dis, 3)))
                if run_epochs%show_epochs == 0:
                    gen_samples, sample_z = show_generated(session=session, output_fake=output_gen_56, n_images=n_images, z_input_56=z_input_56, z_input_112=z_input_112)
#                     img_storage[run_epochs//show_epochs] = gen_samples
#                     latent_storage[run_epochs//show_epochs] = sample_z
                    saver.save(sess = session, save_path=checkpoints, global_step=run_epochs)
                
                run_epochs+=1
                
        

In [ ]:
epochs = 2
batch_size = 32
z_dim = 100
learning_rate = 2e-4
beta1 = 0.5
alpha = 0.2

dataset = 'nki'
marker = 'he'
image_width = 56
image_height = 56
image_channels = 3

global losses_56
global losses_112

name_run = 'h%s_w%s_n%s_3iter' % (image_height, image_width, image_channels)
data_out_path = '/Users/adalbertoclaudioquiros/Documents/Code/UofG/PhD/Cancer_TMA_Generative/data model output/ProGAN/%s' % name_run

data_56 = Data(dataset, marker, patch_h=image_height, patch_w=image_width, n_channels=image_channels, batch_size=batch_size)
data_112 = Data(dataset, marker, patch_h=image_height*2, patch_w=image_width*2, n_channels=image_channels, batch_size=batch_size)
data = [data_56, data_112]

with tf.Graph().as_default():
   train(epochs, batch_size, z_dim, learning_rate, beta1, data, alpha, image_width, image_height, image_channels, data_out_path)

save_loss(losses, data_out_path, dim=20)